In [1]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
class Writer:
    def __init__(self, outdir, start_idx=0):
        self.outdir = Path(outdir)
        self.outdir.mkdir(parents=True, exist_ok=True)
        self.idx = start_idx
    def write(self, X, y):
        save_file = self.outdir / f'{self.idx}.pt'
        torch.save((X, y), save_file)
        self.idx += 1

In [3]:
data_dir = '../Data/CHD_w29_s14_ID_Data/wavelet/'
file_name = 'DoS.pt'
X, y = torch.load(Path(data_dir)/file_name)

In [4]:
def write_to_file(writer, X, y):
    try:
        for xi, yi in zip(X, y):
            writer.write(xi, yi)
    except: return False
    return True

In [5]:
num_splits = 5
test_size = 0.3
outdir = '../Data/CHD_w29_s14_ID_Data/'
train_writers = [Writer(outdir=outdir + f'train/{i + 1}/') for i in range(num_splits)]
val_writers = [Writer(outdir=outdir + f'val/{i + 1}/') for i in range(num_splits)]
sss = StratifiedShuffleSplit(n_splits=num_splits, test_size=test_size)
sss.get_n_splits(X, y)

5

In [8]:
from asyncio import as_completed
from concurrent import futures


with futures.ProcessPoolExecutor(num_splits) as exec:
    todo = []
    for i, (train_idx, test_idx) in enumerate(sss.split(X, y)):
        print(i, len(train_idx), len(test_idx))
        X_train = X[train_idx[:100]]
        y_train = y[train_idx[:100]]
        X_val = X[test_idx[:100]]
        y_val = y[test_idx[:100]]
        future = exec.submit(write_to_file, train_writers[i], X_train, y_train)
        todo.append(future)
        future = exec.submit(write_to_file, val_writers[i], X_val, y_val)
        todo.append(future)

    results = []
    sucess = 0
    fail = 0
    for future in futures.as_completed(todo):
        res = future.result()
        if res:
            sucess += 1
        else:
            fail += 1
print(f'Sucess: {sucess} - Fails: {fail}')

0 183287 78552
Sucess: 2 - Fails: 0
